In [1]:
import numpy as np
import pandas as pd 
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import rdMolDescriptors
import rdkit.Chem as Chem
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingClassifier
from mordred import Calculator, descriptors
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from imblearn.over_sampling import SMOTE

In [2]:
test_data = pd.read_csv('test_II.csv')
train_data = pd.read_csv('train_II.csv')
desc_df = pd.read_csv('descriptorsfromrdkitv1.csv')
test_desc_df = pd.read_csv('testdescriptors.csv')

In [3]:
train_data[['Chemical ID', 'Assay ID']] = train_data['Id'].str.split(';', expand=True)


In [4]:
train_data = train_data.drop('Id', axis=1)

In [5]:
train_data = train_data.rename(columns={'Expected': 'Label'})


In [6]:
# Create a list to store the SMILES strings that do not cause an error
valid_smiles = []

# Iterate over the "Chemical ID" column and store only the valid SMILES strings
for smiles in train_data["Chemical ID"]:
    try:
        mols = Chem.MolFromSmiles(smiles, sanitize=False)
        if mols is not None:
            valid_smiles.append(smiles)
    except:
        continue

# Filter the DataFrame to only include rows with valid SMILES strings
train_data = train_data[train_data["Chemical ID"].isin(valid_smiles)]


In [7]:
train_data['mols'] = train_data['Chemical ID'].apply(lambda x: Chem.MolFromSmiles(x))

[01:00:47] Explicit valence for atom # 1 Si, 8, is greater than permitted
[01:00:48] Explicit valence for atom # 1 Si, 8, is greater than permitted
[01:00:50] Explicit valence for atom # 1 Si, 8, is greater than permitted
[01:00:51] Explicit valence for atom # 1 Si, 8, is greater than permitted
[01:00:52] Explicit valence for atom # 1 Si, 8, is greater than permitted
[01:00:53] Explicit valence for atom # 1 Si, 8, is greater than permitted


In [8]:
train_data = train_data.dropna(subset=['mols'])

In [9]:
test_data[['Chemical ID', 'Assay ID']] = test_data['x'].str.split(';', expand=True)

In [10]:
test_data = test_data.drop('x', axis=1)

In [11]:
# Create a list to store the SMILES strings that do not cause an error
valid_smiles = []

# Iterate over the "Chemical ID" column and store only the valid SMILES strings
for smiles in test_data["Chemical ID"]:
    try:
        mols = Chem.MolFromSmiles(smiles, sanitize=False)
        if mols is not None:
            valid_smiles.append(smiles)
    except:
        continue

# Filter the DataFrame to only include rows with valid SMILES strings
test_data = test_data[test_data["Chemical ID"].isin(valid_smiles)]

In [12]:
test_data['mols'] = test_data['Chemical ID'].apply(lambda x: Chem.MolFromSmiles(x))

In [13]:
desc_df.columns = desc_df.columns.astype(str)

# Replace NaN and infinity values with the mean of the column
desc_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Select only the numeric columns
numeric_cols = desc_df.select_dtypes(include=np.number).columns

# Apply fillna() to the selected numeric columns
desc_df[numeric_cols] = desc_df[numeric_cols].fillna(desc_df[numeric_cols].median())


In [14]:
# Get the fingerprints, assay ID, and labels from the dataframe
train_assay_id = np.array(train_data['Assay ID'].values).reshape(-1, 1)
y_tr = train_data['Label'].to_numpy()

In [15]:
test_desc_df.columns = test_desc_df.columns.astype(str)

# Replace NaN and infinity values with the mean of the column
test_desc_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Select only the numeric columns
numeric_cols = test_desc_df.select_dtypes(include=np.number).columns

# Apply fillna() to the selected numeric columns
test_desc_df[numeric_cols] = test_desc_df[numeric_cols].fillna(test_desc_df[numeric_cols].median())


In [16]:
test_assay_id = np.array(test_data['Assay ID'].values).reshape(-1, 1)

In [17]:
xgb = XGBClassifier(n_estimators=1050)

In [18]:
X_tr = np.concatenate((train_assay_id, desc_df[['Ipc', 'BertzCT', 'MolWt', 'ExactMolWt', 'HeavyAtomMolWt', 'TPSA', 'LabuteASA', 'NumValenceElectrons', 'EState_VSA1', 'MolMR', 'VSA_EState3', 'SMR_VSA1', 'SlogP_VSA2', 'PEOE_VSA1', 'SlogP_VSA11', 'SMR_VSA9', 'PEOE_VSA11', 'EState_VSA10', 'SMR_VSA7', 'VSA_EState2', 'EState_VSA3', 'HeavyAtomCount', 'SlogP_VSA6', 'PEOE_VSA7', 'SlogP_VSA5', 'Chi0', 'Kappa1', 'VSA_EState5', 'PEOE_VSA14', 'Chi0n', 'Chi1', 'Chi0v', 'VSA_EState1', 'PEOE_VSA9', 'PEOE_VSA3', 'NumHAcceptors', 'NOCount', 'NumHeteroatoms', 'SMR_VSA10', 'EState_VSA9', 'PEOE_VSA12', 'Chi1n', 'Kappa2', 'Chi1v', 'PEOE_VSA6', 'fr_phenol_noOrthoHbond', 'fr_phenol', 'fr_Ar_OH', 'NumHDonors', 'NHOHCount', 'Chi2n', 'Chi2v', 'SlogP_VSA3', 'SlogP_VSA1', 'NumRotatableBonds', 'HallKierAlpha', 'VSA_EState7', 'EState_VSA11', 'Chi3n', 'Chi3v', 'EState_VSA8', 'EState_VSA7', 'EState_VSA2', 'EState_VSA5', 'EState_VSA4', 'PEOE_VSA8', 'VSA_EState4', 'Kappa3', 'fr_ether', 'BCUT2D_MWHI', 'EState_VSA6', 'SlogP_VSA12', 'fr_ester', 'fr_C_O_noCOO', 'RingCount', 'fr_C_O', 'fr_benzene', 'NumAromaticCarbocycles', 'Chi4n', 'NumAromaticRings', 'Chi4v', 'PEOE_VSA10', 'MaxAbsEStateIndex', 'SMR_VSA5', 'SlogP_VSA10', 'SlogP_VSA4', 'VSA_EState8', 'SMR_VSA3', 'PEOE_VSA2', 'VSA_EState10', 'SMR_VSA6', 'PEOE_VSA4', 'VSA_EState6', 'SMR_VSA4', 'MolLogP', 'SlogP_VSA8', 'PEOE_VSA5', 'MinEStateIndex', 'SlogP_VSA7', 'PEOE_VSA13', 'FpDensityMorgan3', 'BalabanJ', 'FpDensityMorgan2', 'fr_halogen', 'fr_NH0']]), axis=1) 
X_te = np.concatenate((test_assay_id, test_desc_df[['Ipc', 'BertzCT', 'MolWt', 'ExactMolWt', 'HeavyAtomMolWt', 'TPSA', 'LabuteASA', 'NumValenceElectrons', 'EState_VSA1', 'MolMR', 'VSA_EState3', 'SMR_VSA1', 'SlogP_VSA2', 'PEOE_VSA1', 'SlogP_VSA11', 'SMR_VSA9', 'PEOE_VSA11', 'EState_VSA10', 'SMR_VSA7', 'VSA_EState2', 'EState_VSA3', 'HeavyAtomCount', 'SlogP_VSA6', 'PEOE_VSA7', 'SlogP_VSA5', 'Chi0', 'Kappa1', 'VSA_EState5', 'PEOE_VSA14', 'Chi0n', 'Chi1', 'Chi0v', 'VSA_EState1', 'PEOE_VSA9', 'PEOE_VSA3', 'NumHAcceptors', 'NOCount', 'NumHeteroatoms', 'SMR_VSA10', 'EState_VSA9', 'PEOE_VSA12', 'Chi1n', 'Kappa2', 'Chi1v', 'PEOE_VSA6', 'fr_phenol_noOrthoHbond', 'fr_phenol', 'fr_Ar_OH', 'NumHDonors', 'NHOHCount', 'Chi2n', 'Chi2v', 'SlogP_VSA3', 'SlogP_VSA1', 'NumRotatableBonds', 'HallKierAlpha', 'VSA_EState7', 'EState_VSA11', 'Chi3n', 'Chi3v', 'EState_VSA8', 'EState_VSA7', 'EState_VSA2', 'EState_VSA5', 'EState_VSA4', 'PEOE_VSA8', 'VSA_EState4', 'Kappa3', 'fr_ether', 'BCUT2D_MWHI', 'EState_VSA6', 'SlogP_VSA12', 'fr_ester', 'fr_C_O_noCOO', 'RingCount', 'fr_C_O', 'fr_benzene', 'NumAromaticCarbocycles', 'Chi4n', 'NumAromaticRings', 'Chi4v', 'PEOE_VSA10', 'MaxAbsEStateIndex', 'SMR_VSA5', 'SlogP_VSA10', 'SlogP_VSA4', 'VSA_EState8', 'SMR_VSA3', 'PEOE_VSA2', 'VSA_EState10', 'SMR_VSA6', 'PEOE_VSA4', 'VSA_EState6', 'SMR_VSA4', 'MolLogP', 'SlogP_VSA8', 'PEOE_VSA5', 'MinEStateIndex', 'SlogP_VSA7', 'PEOE_VSA13', 'FpDensityMorgan3', 'BalabanJ', 'FpDensityMorgan2', 'fr_halogen', 'fr_NH0']]), axis=1)

In [19]:
y_tr[y_tr == 2] = 0
y_tr[y_tr == 1] = 1

In [20]:
xgb.fit(X_tr, y_tr)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1050, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [21]:
Z_pred = xgb.predict(X_te)

In [23]:
Z_pred[Z_pred == 0] = 2
Z_pred[Z_pred == 1] = 1

In [24]:
# Combine the SMILES & Assay ID into a single column
test_data['Id'] = test_data['Chemical ID'] + ';' + test_data['Assay ID'].astype(str)

# Create a new dataframe with the Id and Predicted columns
result_data = pd.DataFrame({'Id': test_data['Id'], 'Predicted': Z_pred})

# Save the result data to a csv file
result_data.to_csv('Submission File.csv', index=False)